# 单细胞数据分析与细胞标注测试

## 测试用例说明

本 notebook 测试一个完整的单细胞数据分析流程：

1. **Supervisor** 选择 `code_dev` agent
2. **Code Dev Agent** 执行单细胞数据分析：
   - 生成 Scanpy 代码
   - 运行得到 UMAP 可视化
   - 进行 Leiden 聚类
   - 在每个聚类上做差异表达分析（DE）
   - 返回每个聚类的 top DE 基因
3. **Supervisor** 选择 `tool_caller` agent
4. **Tool Caller Agent** 使用细胞类型注释工具，根据 DE 基因确定细胞类型
5. **Supervisor** 再次选择 `code_dev` agent
6. **Code Dev Agent** 使用确定的细胞类型进行标注

## 预期结果

- UMAP 可视化图
- Leiden 聚类结果
- 每个聚类的细胞类型标注
- 差异表达基因列表


## 1. 导入必要的库


In [1]:
import os
import sys
from pathlib import Path
import json
from typing import Dict, Any

# 添加项目根目录到路径
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# 导入主图
from src.main import graph
from langchain_core.messages import HumanMessage
import os

# proxy_vars = ["HTTP_PROXY", "HTTPS_PROXY", "ALL_PROXY", "http_proxy", "https_proxy", "all_proxy"]

# for var in proxy_vars:
#     if var in os.environ:
#         del os.environ[var]
#         print(f"已清除环境变量: {var}")


/home/luting/miniconda3/envs/langgraph/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[System] Auto-registered tools: ['run_celltype_annotation', 'gene_set_enrichment', 'query_mygene']


## 2. 准备初始状态


In [2]:
# 定义单细胞数据分析任务
user_query = """
请对单细胞数据进行以下分析：
1. 读取数据，进行标准处理，得到聚类结果和 UMAP 可视化，并且对类别进行标注，再进行保存。

数据路径：/home/luting/projects/mas/mas_2/data/pbmc3k.h5ad
输出路径：/home/luting/projects/mas/mas_2/notebooks/result/
"""

# 初始化状态
initial_state = {
    # 基础字段
    "messages": [HumanMessage(content=user_query)],
    "user_query": user_query.strip(),
    # 计划字段
    "plan": [],  # 空列表，Supervisor会自动生成计划
    "current_step_index": 0,
    "current_step_input": None,
    "current_step_expected_output": None,
    "current_step_file_paths": None,
    # 调度控制字段  
    "next_worker": "rag_researcher",  # 初始值，会被 Supervisor 覆盖
    "last_worker": "",
    # 产出字段  
    "final_report": "",
    "code_solution": "",
    "rag_context": "",
    # 交互字段
    "pending_contribution": None,
    "critique_feedback": None,
    "is_approved": False,
}

print("✅ 初始状态准备完成")
print(f"用户查询: {user_query[:100]}...")


✅ 初始状态准备完成
用户查询: 
请对单细胞数据进行以下分析：
1. 读取数据，进行标准处理，得到聚类结果和 UMAP 可视化，并且对类别进行标注，再进行保存。

数据路径：/home/luting/projects/mas/mas...


## 3. 运行主图并跟踪执行流程


In [3]:
# 使用 stream 来跟踪执行流程
execution_steps = []
max_steps = 30  # 设置最大步数，防止无限循环

print("🚀 开始执行主图...")
print("=" * 60)

try:
    for step in graph.stream(initial_state, config={"recursion_limit": max_steps}):
        for node_name, node_state in step.items():
            execution_steps.append({
                "node": node_name,
                "state": node_state.copy()
            })
            
            print(f"\n📌 [{len(execution_steps)}] 执行节点: {node_name}")
            
            # 显示关键状态信息
            if "next_worker" in node_state:
                print(f"   → next_worker: {node_state['next_worker']}")
            if "last_worker" in node_state:
                print(f"   → last_worker: {node_state['last_worker']}")
            if "is_approved" in node_state:
                print(f"   → is_approved: {node_state['is_approved']}")
            if "pending_contribution" in node_state and node_state.get("pending_contribution"):
                pending = node_state["pending_contribution"]
                if isinstance(pending, dict) and "code" in pending:
                    code_preview = pending["code"][:200] if len(pending["code"]) > 200 else pending["code"]
                    print(f"   → 生成的代码预览: {code_preview}...")
            
            # 如果到达 finalize，停止
            if node_name == "finalize":
                print("\n✅ 到达 Finalize 节点，任务完成")
                break
    
    print("\n" + "=" * 60)
    print(f"✅ 执行完成，共执行 {len(execution_steps)} 步")
    
except Exception as e:
    print(f"\n❌ 执行出错: {e}")
    import traceback
    traceback.print_exc()


trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-bec6-7a33-9a31-796e36ce7c50; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-bece-7c73-81b2-2b1621eba919; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-bece-7c73-81b2-2b22b0c3815b; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-becf-7843-bf40-8506b2e64566; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-bed1-78b2-a9bc-47a9ade7d3d1; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-bed1-78b2-a9bc-47b806577170


🚀 开始执行主图...
--- [Supervisor] 正在做调度决策 ---
--- [Supervisor] 正在生成执行计划 ---


INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-e97d-7a60-ad49-d1be74a23a18; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-e97e-7960-8792-d142cf6f533d; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-e97f-7f40-b638-7d49856c2a77; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-e97f-7f40-b638-7d51b178ef6c; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-e980-7d31-a69f-82f45b864f1b; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-e981-77c0-a561-0f6912369856; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-bece-7c73-81b2-2b1621eba919; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-bece-7c73-81b2-2b22b0c3815b; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-becf-7843-bf40-8506b2e64566; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-bed1-78b2-a9bc-47a9ade7d3d1; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id

  --> 计划生成成功，共 7 个步骤
     步骤 1: 读取数据
     步骤 2: 标准化处理
     步骤 3: 特征选择
     步骤 4: 降维处理
     步骤 5: 聚类分析
     步骤 6: 类别标注
     步骤 7: 结果保存
  --> 当前执行步骤 1/7: 读取数据
  --> 派遣到 code_dev

📌 [1] 执行节点: supervisor
   → next_worker: code_dev
   → last_worker: 
   → is_approved: False
--- [Code Dev] 正在生成代码 (迭代 1) ---
  --> 从用户查询中解析到数据路径: /home/luting/projects/mas/mas_2/data/pbmc3k.h5ad
  --> 从用户查询中解析到结果路径: /home/luting/projects/mas/mas_2/notebooks/result/


INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.agents.code_dev.executor:数据目录挂载: /home/luting/projects/mas/mas_2/data -> /app/data
INFO:src.agents.code_dev.executor:构建Docker镜像: code-executor-70f23f02
INFO:src.agents.code_dev.executor:运行容器，镜像: code-executor-70f23f02
trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-ef6c-7aa1-b94e-6e571a3222a5; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-ef6d-76f1-b45d-ec092b063767; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-e980-7d31-a69f-82f45b864f1b; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-e981-77c0-a561-0f6912369856


...

INFO:src.agents.code_dev.executor:运行容器，镜像: code-executor-b305b179
trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3efa-d0bc-73d2-916b-58e0c0c28544; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3efa-d0bd-7552-8493-d1fa861774c4; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3efa-d0c1-7700-883f-585bdd57aa83; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3efa-d0c1-7700-883f-5860eb11284a; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3efa-d0c3-7702-ab61-b79eae9b213d; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3efa-d0c3-7702-ab61-b7a13f948d29; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3ef8-d7dd-7d30-9a1c-60193cff7c71; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3ef6-919a-7bf2-b7a1-a3a291a40243; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3ef6-919b-7ab1-b3b9-f8f1ff7576d3


【Docker代码执行日志】: DEBUG: Checking /app/data contents...
DEBUG: Files in /app/data: ['step_2_normalized_data.h5ad', 'step_3_highly_variable_genes.txt', 'step_2_preprocessed_data.h5ad', 'step_3_selected_features.h5ad', 'step_1_data.h5ad']
DEBUG: Files in /app/output: ['step_2_normalized_data.h5ad', 'step_3_highly_variable_genes.txt', 'step_2_preprocessed_data.h5ad', 'step_3_selected_features.h5ad', 'step_1_data.h5ad']
computing neighbors
/app/code.py:35: UserWarning: You’re trying to run this on 1930 dimensions of `.X`, if you really want this, set `use_rep='X'`.
         Falling back to preprocessing with `sc.pp.pca` and default params.
  sc.pp.neighbors(adata)
computing PCA
    with n_comps=50
    finished (0:00:00)
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:17)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm)
    'umap', UMAP paramet

INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


  --> 审核驳回！意见: [FAIL]
CRITICAL ISSUE: 代码中未进行聚类分析，导致聚类数量为0，无法正确生成带有类别标注的UMAP图。
SUGGESTION: 在调用`sc.tl.umap`之前，应先运行`sc.tl.leiden`或`sc.tl.louvain`进行聚类，并确保在UMAP可视化时使用`color='leiden'`参数对类别进行标注。
  --> 审核驳回，返回 code_dev 重做

📌 [30] 执行节点: critic
   → next_worker: code_dev
   → last_worker: code_dev
   → is_approved: False
   → 生成的代码预览: import scanpy as sc
import sys

# 读取数据
adata = sc.read_h5ad("/app/data/step_3_selected_features.h5ad")

# 构建邻域图
sc.pp.neighbors(adata)

# 进行UMAP降维
sc.tl.umap(adata)

# 绘制UMAP图
sc.pl.umap(adata, show=F...

❌ 执行出错: Recursion limit of 30 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/GRAPH_RECURSION_LIMIT


Traceback (most recent call last):
  File "/tmp/ipykernel_2644147/1463509198.py", line 9, in <module>
    for step in graph.stream(initial_state, config={"recursion_limit": max_steps}):
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/luting/miniconda3/envs/langgraph/lib/python3.12/site-packages/langgraph/pregel/main.py", line 2674, in stream
    raise GraphRecursionError(msg)
langgraph.errors.GraphRecursionError: Recursion limit of 30 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/GRAPH_RECURSION_LIMIT


## 4. 提取最终结果


In [4]:
# 获取最终状态
if execution_steps:
    final_state = execution_steps[-1]["state"]
    
    print("📊 最终状态摘要:")
    print("=" * 60)
    
    # 显示最终答案
    if "final_answer" in final_state:
        print(f"\n📝 最终答案:\n{final_state['final_answer']}")
    
    # 显示代码解决方案
    if final_state.get("code_solution"):
        print(f"\n💻 代码解决方案:\n{final_state['code_solution'][:500]}...")
    
    # 显示 RAG 上下文
    if final_state.get("rag_context"):
        print(f"\n📚 RAG 上下文:\n{final_state['rag_context'][:300]}...")
    
    # 显示最终报告
    if final_state.get("final_report"):
        print(f"\n📄 最终报告:\n{final_state['final_report'][:500]}...")
    
    print("\n" + "=" * 60)
else:
    print("⚠️ 没有执行步骤记录")


📊 最终状态摘要:



## 5. 分析执行流程


In [5]:
# 分析执行流程
print("🔄 执行流程分析:")
print("=" * 60)

node_sequence = [step["node"] for step in execution_steps]
print(f"\n节点执行顺序: {' -> '.join(node_sequence)}")

# 统计各节点执行次数
from collections import Counter
node_counts = Counter(node_sequence)
print(f"\n各节点执行次数:")
for node, count in node_counts.items():
    print(f"  - {node}: {count} 次")

# 查找关键步骤
print("\n🔍 关键步骤:")
for i, step in enumerate(execution_steps):
    node = step["node"]
    state = step["state"]
    
    # Code Dev 生成代码
    if node == "code_dev" and state.get("pending_contribution"):
        pending = state["pending_contribution"]
        if isinstance(pending, dict) and "code" in pending:
            print(f"\n  [{i+1}] Code Dev 生成代码 (步骤 {i+1})")
            print(f"      代码长度: {len(pending['code'])} 字符")
    
    # Tool Caller 调用工具
    if node == "tool_caller" and state.get("tool_name"):
        print(f"\n  [{i+1}] Tool Caller 调用工具: {state['tool_name']} (步骤 {i+1})")
        if state.get("tool_result"):
            result_preview = str(state["tool_result"])[:200]
            print(f"      结果预览: {result_preview}...")
    
    # Critic 审核结果
    if node == "critic":
        is_approved = state.get("is_approved", False)
        status = "✅ 通过" if is_approved else "❌ 驳回"
        print(f"\n  [{i+1}] Critic 审核: {status} (步骤 {i+1})")
        if state.get("critique_feedback"):
            feedback = state["critique_feedback"][:200]
            print(f"      反馈: {feedback}...")


trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3efa-d6d6-7ce0-b241-65fed6672372; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3efa-d0c1-7700-883f-585bdd57aa83; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3efa-d0c1-7700-883f-5860eb11284a; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3efa-d0c3-7702-ab61-b79eae9b213d; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3efa-d0c3-7702-ab61-b7a13f948d29; trace=019c3eee-bec6-7a33-9a31-796e36ce7c50,id=019c3eee-bec6-7a33-9a31-796e36ce7c50


🔄 执行流程分析:

节点执行顺序: supervisor -> code_dev -> critic -> code_dev -> critic -> code_dev -> critic -> code_dev -> critic -> code_dev -> critic -> code_dev -> critic -> code_dev -> critic -> code_dev -> critic -> supervisor -> code_dev -> critic -> supervisor -> code_dev -> critic -> code_dev -> critic -> supervisor -> code_dev -> critic -> code_dev -> critic

各节点执行次数:
  - supervisor: 4 次
  - code_dev: 13 次
  - critic: 13 次

🔍 关键步骤:

  [2] Code Dev 生成代码 (步骤 2)
      代码长度: 319 字符

  [3] Critic 审核: ❌ 驳回 (步骤 3)
      反馈: [FAIL]
CRITICAL ISSUE: 代码中尝试获取 `adata.obs['leiden']` 的类别数量，但该列在数据中不存在，导致聚类数量为 0，这表明数据未正确加载或未进行聚类分析。此外，输出路径 `/app/output/step_1_data.h5ad` 与用户指定的输出路径 `/home/luting/projects/mas/mas_2/notebooks/result/`...

  [4] Code Dev 生成代码 (步骤 4)
      代码长度: 110 字符

  [5] Critic 审核: ❌ 驳回 (步骤 5)
      反馈: [FAIL]
CRITICAL ISSUE: 代码中读取的数据路径为 "/app/data/pbmc3k.h5ad"，但用户提供的数据路径为 "/home/luting/projects/mas/mas_2/data/pbmc3k.h5ad"，路径不一致可能导致文件无法正确加载。
SUGGESTION: 修改代码中的数据路径为用户提供的路径，确保文件能够被正确读取。...



## 6. 提取代码和结果


In [6]:
# 提取所有生成的代码
generated_codes = []

for step in execution_steps:
    state = step["state"]
    
    # 从 pending_contribution 提取代码
    if state.get("pending_contribution"):
        pending = state["pending_contribution"]
        if isinstance(pending, dict) and "code" in pending:
            generated_codes.append({
                "step": step["node"],
                "code": pending["code"]
            })
    
    # 从 code_solution 提取代码
    if state.get("code_solution"):
        generated_codes.append({
            "step": "code_solution",
            "code": state["code_solution"]
        })

print(f"📝 找到 {len(generated_codes)} 段生成的代码\n")

for i, code_info in enumerate(generated_codes, 1):
    print(f"代码段 {i} (来源: {code_info['step']}):")
    print("-" * 60)
    print(code_info["code"][:500])
    if len(code_info["code"]) > 500:
        print("...")
    print()


📝 找到 23 段生成的代码

代码段 1 (来源: code_dev):
------------------------------------------------------------
import scanpy as sc

adata = sc.read_h5ad("/app/data/pbmc3k.h5ad")
adata.write("/app/output/step_1_data.h5ad")

n_clusters = len(adata.obs['leiden'].cat.categories) if 'leiden' in adata.obs else 0
analysis_summary = f"细胞总数：{adata.n_obs}，基因总数：{adata.n_vars}，聚类数量：{n_clusters}"
print(f"===RESULT==={analysis_summary}===")

代码段 2 (来源: critic):
------------------------------------------------------------
import scanpy as sc

adata = sc.read_h5ad("/app/data/pbmc3k.h5ad")
adata.write("/app/output/step_1_data.h5ad")

n_clusters = len(adata.obs['leiden'].cat.categories) if 'leiden' in adata.obs else 0
analysis_summary = f"细胞总数：{adata.n_obs}，基因总数：{adata.n_vars}，聚类数量：{n_clusters}"
print(f"===RESULT==={analysis_summary}===")

代码段 3 (来源: code_dev):
------------------------------------------------------------
import scanpy as sc

adata = sc.read_h5ad("/app/data/pbmc3k.h5ad")
adata.write("/app/output/ste

## 7. 提取细胞类型标注结果


In [7]:
# 查找 Tool Caller 的细胞类型注释结果
celltype_results = []

for step in execution_steps:
    state = step["state"]
    
    # 查找 tool_caller 的结果
    if step["node"] == "tool_caller":
        tool_result = state.get("tool_result")
        if tool_result and isinstance(tool_result, dict):
            # 检查是否是细胞类型注释结果
            if "final_prediction" in tool_result:
                celltype_results.append({
                    "step": len(celltype_results) + 1,
                    "prediction": tool_result.get("final_prediction"),
                    "decision_logic": tool_result.get("decision_logic"),
                    "details": tool_result.get("details", {})
                })

if celltype_results:
    print("🔬 细胞类型注释结果:")
    print("=" * 60)
    
    for result in celltype_results:
        print(f"\n结果 {result['step']}:")
        print(f"  预测细胞类型: {result['prediction']}")
        print(f"  决策逻辑: {result['decision_logic']}")
        
        if result.get("details"):
            details = result["details"]
            if "enrichr" in details:
                enrichr = details["enrichr"]
                print(f"  Enrichr 结果: {enrichr.get('label', 'N/A')} (置信度: {enrichr.get('confidence', 0):.2f})")
            if "llm_expert" in details:
                llm = details["llm_expert"]
                print(f"  LLM 专家结果: {llm.get('label', 'N/A')} (置信度: {llm.get('confidence', 0):.2f})")
else:
    print("⚠️ 未找到细胞类型注释结果")


⚠️ 未找到细胞类型注释结果


## 8. 保存结果到文件


In [8]:
# 保存执行结果
output_dir = Path("../results")
output_dir.mkdir(exist_ok=True)

# 保存执行步骤摘要
summary = {
    "total_steps": len(execution_steps),
    "node_sequence": [step["node"] for step in execution_steps],
    "final_state": {
        "final_answer": final_state.get("final_answer", "") if execution_steps else "",
        "code_solution": final_state.get("code_solution", "")[:1000] if execution_steps else "",  # 限制长度
        "rag_context": final_state.get("rag_context", "")[:500] if execution_steps else "",
        "final_report": final_state.get("final_report", "")[:500] if execution_steps else ""
    },
    "generated_codes_count": len(generated_codes),
    "celltype_results_count": len(celltype_results)
}

output_file = output_dir / "single_cell_analysis_result.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

print(f"✅ 结果已保存到: {output_file}")

# 保存生成的代码
if generated_codes:
    code_file = output_dir / "generated_code.py"
    with open(code_file, "w", encoding="utf-8") as f:
        for i, code_info in enumerate(generated_codes, 1):
            f.write(f"# Code segment {i} (from {code_info['step']})\n")
            f.write("# " + "="*60 + "\n")
            f.write(code_info["code"])
            f.write("\n\n")
    print(f"✅ 代码已保存到: {code_file}")


✅ 结果已保存到: ../results/single_cell_analysis_result.json
✅ 代码已保存到: ../results/generated_code.py
